In [102]:
# This code trains models to predict target assessments (PHQ-9, GAD-7) from semntaic location and reason data

import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from calculate_confusion_matrix import calculate_confusion_matrix
import time

save_results = False
do_stratify = False

with open('location_assessment.dat') as f:
    data = pickle.load(f)
f.close

feature = data[['Home', 'Shop or Store', 'Food (Restaurant, Cafe)',\
                "Another's Home", 'Professional or Medical Office', 'Work',\
                'Arts & Entertainment (Theater, Music Venue, Etc.)',\
                'Outdoors & Recreation', 'Nightlife Spot (Bar, Club)', 'Vehicle',\
                'DUR Home', 'DUR Shop or Store', 'DUR Food (Restaurant, Cafe)',\
                "DUR Another's Home", 'DUR Professional or Medical Office',\
                'DUR Work', 'DUR Arts & Entertainment (Theater, Music Venue, Etc.)',\
                'DUR Outdoors & Recreation', 'DUR Nightlife Spot (Bar, Club)',\
                'DUR Vehicle', 'home', 'errand', 'dining', 'socialize', 'work',\
                'entertainment', 'dining,socialize', 'travelling / traffic',\
                'exercise', 'entertainment,socialize', 'DUR home', 'DUR errand',\
                'DUR dining', 'DUR socialize', 'DUR work', 'DUR entertainment',\
                'DUR dining,socialize', 'DUR travelling / traffic', 'DUR exercise',\
                'DUR entertainment,socialize']].astype(float)

target = data['SPIN W3']

# remove nans from target
ind = np.where(np.array(pd.isnull(target)))[0]
target = target.drop(ind)
feature = feature.drop(ind, axis=0)
target = target.reset_index(drop=True)
feature = feature.reset_index(drop=True)

# classification
target.loc[target<19] = 0
target.loc[target>=19] = 1

confs = []
aucs = []
labels = []

y_pred = np.zeros(feature.shape[0])

for i in range(feature.shape[0]):
    
    print i,
    
    # training set
    j_range = range(feature.shape[0])
    j_range.pop(i)
    
    x_train = feature.loc[j_range,:]
    y_train = target.loc[j_range]
    
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    
#     if do_stratify:
#         x_train, y_train = stratify(x_train,y_train)
    
    # test set
    x_test = pd.DataFrame(feature.loc[i]).transpose()
    y_test = target.loc[i]
    
    # train (layer 1)
    #eta_list = np.array([0.05]*200+[0.02]*200+[0.01]*200)
    gbm = xgb.XGBClassifier(max_depth=1, n_estimators=1000, learning_rate=0.01, nthread=12, subsample=1,\
                               max_delta_step=0).fit(x_train, y_train)
    
    # train performance
#     y_pred = gbm.predict(x_train)
#     conf_train, roc_auc_train = calculate_confusion_matrix(y_pred, y_train)

    # test (layer 1)
    y_pred[i] = gbm.predict(x_test)
    
    # test performance
    
    
    # foursquare performance
    #conf_fsq, roc_auc_fsq = calculate_confusion_matrix(state_fsq_all[i], y_test)
    

conf, roc_auc = calculate_confusion_matrix(y_pred, np.array(target))

print roc_auc[0]
print conf

# saving the results
if save_results:
    with open('auc_location_sensor_fsq.dat','w') as f:
        #pickle.dump([aucs, confs, labels, aucs_fsq, confs_fsq], f)
        pickle.dump([aucs, confs, labels], f)
    f.close()





 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 0.507175472929
[[ 17.  57.]
 [ 25.  89.]]


In [98]:
# feature.to_csv("x.csv")
# target.to_csv("y.csv")